## Load Libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# distributions
import scipy.integrate as integrate
from scipy.integrate import quad, quad_vec
from scipy.stats import invgamma
from scipy.stats import levy
from scipy.special import gamma, erfinv

import time
from datetime import datetime, timedelta

import sys
import importlib # ref: https://askubuntu.com/questions/144698/find-a-file-by-name-using-command-line
import seaborn as sns

import os

In [2]:
sys.path.append('../tools')
import tools

In [3]:
importlib.reload(tools)

<module 'tools' from '../tools/tools.py'>

## Download Data

In [4]:
df = pd.read_csv('data/apple_cleaned.csv')

## Estimate Spot Volatility

In [5]:
# decide whether you use sample or not
sample = False

if sample:
    data = df[df['date']< '2017-01-30']
    
else:
    data = df.copy()

In [6]:
# Plot style
widths = [1,1,1,1,2]
types = ['solid','dashed','dashdot','dotted', 'dotted']

In [7]:
gs =  [ 0.5, 1, 2, 4]

In [156]:
np.nanmin(data['delta_ts'])

1.0000000000000002e-06

### By bins

#### Wide 0

In [9]:
ts = np.array([0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10])

#### Pdf and Cdf

In [157]:
delta = (3*8*n**(1/5))/(2*(max(ts) - min(ts)))

In [158]:
delta

38.997492424147275

In [189]:
##### check the for loop, is it actually doing what it is supposed to d
sigma_hats_ts = []
f_hats_ts = []
F_hats_ts= []

constant = 400
n = np.shape(data)[0]
delta = (np.sqrt(n)/2)*constant
#delta = (3*8*n**(1/5))/(2*(max(ts) - min(ts)))

# subset the dataframe
t_g_ws = np.array(data['delta_ts']) 
    
# for each t
for (t, i) in zip(ts, range(0, len(ts))):
    

    # start_time for current iteration
    start_time = time.time()
    

    # calculate f_hat
    f_hat = tools.f_hat_t(t, t_g_ws, delta)

    # calculate F_hat
    F_hat = tools.F_hat_t(t, t_g_ws, delta)

    
    # store the estimates
    f_hats_ts.append(f_hat)
    F_hats_ts.append(F_hat)

    # show the end time
    end_time = time.time()  
    
    
    if i in [0,1, 10, 100, 500, 750, 1000, 5000, 10000]:
        print(f"Time : {end_time - start_time} for t = {t} for i = {i}")

Time : 19.68779706954956 for t = 1e-07 for i = 0
Time : 21.29036593437195 for t = 1e-06 for i = 1


#### Spot Volatility

In [163]:
#a_n = (8*(n)**(0.2))/(max(ts) - min(ts))

In [190]:
a_n = (2/3)*delta

In [191]:
sigma_hats_gs = []
std_gs = []

for g_i, g in zip(range(0, len(gs)), gs):
    
    sigma_hats_ts = []
    std_ts = []
    
    # for each t
    for i, t in zip(range(0, len(ts)),ts):
        
        sigma_hats_t = tools.sigma_hat_t(f_hats_ts[i],  F_hats_ts[i], g)[0]
        sigma_hats_ts.append(sigma_hats_t)
        
        # calculate standard deviation
        std = np.sqrt((n/a_n)*(tools.fgW( tools.PgW_inv(F_hats_ts[i], g), g)/f_hats_ts[i]))**(-1)
        std_ts.append(std)
    
    sigma_hats_gs.append(sigma_hats_ts)
    std_gs.append(std_ts)
    
    
    

<ipython-input-191-3cdd944aad3e>:16: RuntimeWarning: divide by zero encountered in double_scalars
  std = np.sqrt((n/a_n)*(tools.fgW( tools.PgW_inv(F_hats_ts[i], g), g)/f_hats_ts[i]))**(-1)


#### Summary Stats

In [166]:
summary_stats = [np.nanmean]

# list to store summaries
summaries = []

# for each t
for i, t in zip(range(0, len(ts)),ts):
    
    
    # calculate summary stats
    f_stats = tools.estimation_summary(f_hats_ts[i], summary_stats, rounding = 10)
    F_stats = tools.estimation_summary(F_hats_ts[i], summary_stats, rounding = 10)
    # create a dataframe
    summary_t = pd.DataFrame([f_stats, F_stats],columns =  ['Mean'],
                            index = [np.tile(t, 2) , ['$\widehat{f}$', '$\widehat{F}$']])
    
    summary_t.index.set_names(['t', 'Statistics'], inplace = True)
    
    # store the summary
    summaries.append(summary_t)

    


In [167]:
summaries_pdf_cdf = pd.concat(summaries).reset_index().pivot(index=['t'], columns='Statistics', values='Mean')

In [168]:
columns_titles = ["$\widehat{f}$","$\widehat{F}$"]
summaries_pdf_cdf = summaries_pdf_cdf.reindex(columns=columns_titles)

summaries_pdf_cdf = summaries_pdf_cdf.reset_index()
summaries_pdf_cdf.columns.name = ''

In [170]:
print(summaries_pdf_cdf.to_latex( caption = 'Estimated pdf and cdf of the first-hitting times in seconds for Apple in 2017 and several t',
                        float_format="%.2f",
                        formatters = {'t': "{:.2E}".format,
                                      'Mean': "{:.2E}".format,
                                      '$\widehat{f}$': "{:.2E}".format,
                                      '$\widehat{F}$': "{:.2E}".format,
                                      
                                      
#                                       'Median': "{:.2E}".format,
#                                       'Std': "{:.2E}".format,
#                                       '5.0%': "{:.2E}".format,
#                                       '95%': "{:.2E}".format
                                     },
                        index = False,
                        position = 'H',
                        label = 'empirics_table_estpdfcdf',
                        #longtable = True,
                        escape=False))

\begin{table}[H]
\centering
\caption{Estimated pdf and cdf of the first-hitting times in seconds for Apple in 2017 and several t}
\label{empirics_table_estpdfcdf}
\begin{tabular}{rrr}
\toprule
       t & $\widehat{f}$ & $\widehat{F}$ \\
\midrule
1.00E-07 &      2.52E+01 &      3.11E-01 \\
1.00E-06 &      2.52E+01 &      3.11E-01 \\
1.00E-05 &      2.52E+01 &      3.11E-01 \\
1.00E-04 &      2.52E+01 &      3.13E-01 \\
1.00E-03 &      2.48E+01 &      3.36E-01 \\
1.00E-02 &      1.75E+01 &      5.28E-01 \\
1.00E-01 &      7.01E-01 &      7.61E-01 \\
1.00E+00 &      7.63E-02 &      9.55E-01 \\
1.00E+01 &      3.85E-06 &      1.00E+00 \\
\bottomrule
\end{tabular}
\end{table}



<ipython-input-170-4f0667ccdca4>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(summaries_pdf_cdf.to_latex( caption = 'Estimated pdf and cdf of the first-hitting times in seconds for Apple in 2017 and several t',


In [171]:
summary_stats = [np.nanmean]
summaries = []

# Iterate over groups
for g_i, g in enumerate(gs):
    sigma_hats_g = sigma_hats_gs[g_i]

    # Iterate over time periods
    for i, t in enumerate(ts):
        sigma_hats = sigma_hats_g[i]

        # Calculate summary statistics
        sigma_hats_stats = tools.estimation_summary(sigma_hats, summary_stats, rounding=10)

        # Create a dataframe
        summary_t = pd.DataFrame([sigma_hats_stats],
                                 columns=['Mean'],
                                 index=pd.MultiIndex.from_tuples([(g, t, '$\widehat{\sigma}_{t}$')],
                                                                 names=['g', 't', 'Statistics']))

        summaries.append(summary_t)

In [181]:
pd.concat(summaries).reset_index().pivot(index=['t', 'Statistics'], columns='g', values='Mean')

,g,0.5,1.0,2.0,4.0
t,Statistics,,,,
1.000000e-07,Standard Deviation,0.004264,0.008527,0.017055,0.034110
1.000000e-06,Standard Deviation,0.004264,0.008528,0.017056,0.034112
1.000000e-05,Standard Deviation,0.004266,0.008533,0.017066,0.034132
1.000000e-04,Standard Deviation,0.004287,0.008574,0.017148,0.034296
1.000000e-03,Standard Deviation,0.004463,0.008926,0.017852,0.035704
1.000000e-02,Standard Deviation,0.006189,0.012378,0.024756,0.049512
1.000000e-01,Standard Deviation,0.003421,0.006842,0.013684,0.027367
1.000000e+00,Standard Deviation,0.013609,0.027217,0.054434,0.108869
1.000000e+01,Standard Deviation,16.078605,32.157210,64.314420,128.628840


In [182]:
summaries_sigma = pd.concat(summaries).reset_index().pivot(index=['t', 'Statistics'], columns='g', values='Mean')

In [183]:
print(summaries_sigma.to_latex( caption = 'Estimated instantaneous variance of the first-hitting times in seconds for Apple in 2017 and several $t$',
                        float_format="%.2f",
                        formatters = {'t': "{:.2E}".format,
                                      '0.5': "{:.2E}".format,
                                      '1.0': "{:.2E}".format,
                                      '2.0': "{:.2E}".format,
                                      '4.0': "{:.2E}".format,
                                      '1.0': "{:.2E}".format,
                                      },
                        position = 'H',
                        #longtable = True,
                        escape=False,
                        label = 'table:est_spt_vol'))

\begin{table}[H]
\centering
\caption{Estimated instantaneous variance of the first-hitting times in seconds for Apple in 2017 and several $t$}
\label{table:est_spt_vol}
\begin{tabular}{llrrrr}
\toprule
             & g &   0.5 &   1.0 &   2.0 &    4.0 \\
t & Statistics &       &       &       &        \\
\midrule
1.000000e-07 & Standard Deviation &  0.00 &  0.01 &  0.02 &   0.03 \\
1.000000e-06 & Standard Deviation &  0.00 &  0.01 &  0.02 &   0.03 \\
1.000000e-05 & Standard Deviation &  0.00 &  0.01 &  0.02 &   0.03 \\
1.000000e-04 & Standard Deviation &  0.00 &  0.01 &  0.02 &   0.03 \\
1.000000e-03 & Standard Deviation &  0.00 &  0.01 &  0.02 &   0.04 \\
1.000000e-02 & Standard Deviation &  0.01 &  0.01 &  0.02 &   0.05 \\
1.000000e-01 & Standard Deviation &  0.00 &  0.01 &  0.01 &   0.03 \\
1.000000e+00 & Standard Deviation &  0.01 &  0.03 &  0.05 &   0.11 \\
1.000000e+01 & Standard Deviation & 16.08 & 32.16 & 64.31 & 128.63 \\
\bottomrule
\end{tabular}
\end{table}



<ipython-input-183-7e042212c3ba>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(summaries_sigma.to_latex( caption = 'Estimated instantaneous variance of the first-hitting times in seconds for Apple in 2017 and several $t$',


In [184]:
summary_stats = [np.nanmean]
summaries = []

# Iterate over groups
for g_i, g in enumerate(gs):
    std_g = std_gs[g_i]

    # Iterate over time periods
    for i, t in enumerate(ts):
        std = std_g[i]

        # Calculate summary statistics
        std_stats = tools.estimation_summary(std, summary_stats, rounding=10)

        # Create a dataframe
        summary_t = pd.DataFrame([std_stats],
                                 columns=['Mean'],
                                 index=pd.MultiIndex.from_tuples([(g, t, 'Standard Deviation')],
                                                                 names=['g', 't', 'Statistics']))

        summaries.append(summary_t)

In [185]:
summaries_sigma_std = pd.concat(summaries).reset_index().pivot(index=['t', 'Statistics'], columns='g', values='Mean')

In [186]:
summaries_sigma_std.index

MultiIndex([( 1e-07, 'Standard Deviation'),
            ( 1e-06, 'Standard Deviation'),
            ( 1e-05, 'Standard Deviation'),
            (0.0001, 'Standard Deviation'),
            ( 0.001, 'Standard Deviation'),
            (  0.01, 'Standard Deviation'),
            (   0.1, 'Standard Deviation'),
            (   1.0, 'Standard Deviation'),
            (  10.0, 'Standard Deviation')],
           names=['t', 'Statistics'])

In [187]:
summaries_sigma_std

,g,0.5,1.0,2.0,4.0
t,Statistics,,,,
1.000000e-07,Standard Deviation,0.004264,0.008527,0.017055,0.034110
1.000000e-06,Standard Deviation,0.004264,0.008528,0.017056,0.034112
1.000000e-05,Standard Deviation,0.004266,0.008533,0.017066,0.034132
1.000000e-04,Standard Deviation,0.004287,0.008574,0.017148,0.034296
1.000000e-03,Standard Deviation,0.004463,0.008926,0.017852,0.035704
1.000000e-02,Standard Deviation,0.006189,0.012378,0.024756,0.049512
1.000000e-01,Standard Deviation,0.003421,0.006842,0.013684,0.027367
1.000000e+00,Standard Deviation,0.013609,0.027217,0.054434,0.108869
1.000000e+01,Standard Deviation,16.078605,32.157210,64.314420,128.628840


In [188]:
print(summaries_sigma_std.sort_index(level = 1, sort_remaining = 0).to_latex( caption = 'Estimated instantaneous standard deviation of the first-hitting times in seconds for Apple in 2017 and several $t$',
                        float_format="%.2f",
                        formatters = {
                                      
                                      't': "{:.2E}".format,
                                      '0.5': "{:.2E}".format,
                                      '1.0': "{:.2E}".format,
                                      '2.0': "{:.2E}".format,
                                      '4.0': "{:.2E}".format,
                                      '1.0': "{:.2E}".format,
                                      },
                        position = 'H',
                        #longtable = True,
                        escape=False,
                        label = 'table:est_spt_std'))

\begin{table}[H]
\centering
\caption{Estimated instantaneous standard deviation of the first-hitting times in seconds for Apple in 2017 and several $t$}
\label{table:est_spt_std}
\begin{tabular}{llrrrr}
\toprule
             & g &   0.5 &   1.0 &   2.0 &    4.0 \\
t & Statistics &       &       &       &        \\
\midrule
1.000000e-07 & Standard Deviation &  0.00 &  0.01 &  0.02 &   0.03 \\
1.000000e-06 & Standard Deviation &  0.00 &  0.01 &  0.02 &   0.03 \\
1.000000e-05 & Standard Deviation &  0.00 &  0.01 &  0.02 &   0.03 \\
1.000000e-04 & Standard Deviation &  0.00 &  0.01 &  0.02 &   0.03 \\
1.000000e-03 & Standard Deviation &  0.00 &  0.01 &  0.02 &   0.04 \\
1.000000e-02 & Standard Deviation &  0.01 &  0.01 &  0.02 &   0.05 \\
1.000000e-01 & Standard Deviation &  0.00 &  0.01 &  0.01 &   0.03 \\
1.000000e+00 & Standard Deviation &  0.01 &  0.03 &  0.05 &   0.11 \\
1.000000e+01 & Standard Deviation & 16.08 & 32.16 & 64.31 & 128.63 \\
\bottomrule
\end{tabular}
\end{table}



<ipython-input-188-f62c4d675a07>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(summaries_sigma_std.sort_index(level = 1, sort_remaining = 0).to_latex( caption = 'Estimated instantaneous standard deviation of the first-hitting times in seconds for Apple in 2017 and several $t$',


In [ ]:
#### todos
# check delta, based on new a_n
# find which g to use, based on some algorithm
# - focus on Levy
# - fitted parameter for simulations in Levy
# - results in the middle, like around 1 etc, not too close to zero

# - (normaliize the plots by volatility), devide by the volatility
# it does not change with g too much, when normalized
# smart (idea), rigorous, in line with literature!

# time-varying alpha,
# try time-varying alpha
# and then different gs

In [ ]:

## fit the parameter model to the data, and then we use this,
## and then choose g = sqrt(c), then choose different g,
## we will see but we are not sure
##
